![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Agents Lab Notebook v1.0.0
This notebook contains steps and code to demonstrate the use of agents
configured in Agent Lab in watsonx.ai. It introduces Python API commands
for authentication using API key and invoking a LangGraph agent with a watsonx chat model.

**Note:** Notebook code generated using Agent Lab will execute successfully.
If code is modified or reordered, there is no guarantee it will successfully execute.
For details, see: <a href="/docs/content/wsj/analyze-data/fm-prompt-save.html?context=wx" target="_blank">Saving your work in Agent Lab as a notebook.</a>

Some familiarity with Python is helpful. This notebook uses Python 3.11.

## Notebook goals
The learning goals of this notebook are:

* Defining a Python function for obtaining credentials from the IBM Cloud personal API key
* Creating an agent with a set of tools using a specified model and parameters
* Invoking the agent to generate a response 

# Setup

In [ ]:
# import dependencies
from langchain_ibm import ChatWatsonx
from ibm_watsonx_ai import APIClient
from langchain_core.messages import AIMessage, HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from ibm_watsonx_ai.foundation_models.utils import Tool, Toolkit
import json
import requests

## watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud personal API key. For details, see
<a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank">documentation</a>.


In [ ]:
import os
import getpass

def get_credentials():
	return {
		"url" : "https://us-south.ml.cloud.ibm.com",
		"apikey" : getpass.getpass("Please enter your api key (hit enter): ")
	}

def get_bearer_token():
    url = "https://iam.cloud.ibm.com/identity/token"
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    data = f"grant_type=urn:ibm:params:oauth:grant-type:apikey&apikey={credentials['apikey']}"

    response = requests.post(url, headers=headers, data=data)
    return response.json().get("access_token")

credentials = get_credentials()

# Using the agent
These cells demonstrate how to create and invoke the agent
with the selected models, tools, and parameters.

## Defining the model id
We need to specify model id that will be used for inferencing:

In [ ]:
model_id = "ibm/granite-3-3-8b-instruct"

## Defining the model parameters
We need to provide a set of model parameters that will influence the
result:

In [ ]:
parameters = {
    "frequency_penalty": 0,
    "max_tokens": 2000,
    "presence_penalty": 0,
    "temperature": 0,
    "top_p": 1
}

## Defining the project id or space id
The API requires project id or space id that provides the context for the call. We will obtain
the id from the project or space in which this notebook runs:

In [ ]:
project_id = os.getenv("PROJECT_ID")
space_id = os.getenv("SPACE_ID")


## Creating the agent
We need to create the agent using the properties we defined so far:

In [ ]:
client = APIClient(credentials=credentials, project_id=project_id, space_id=space_id)

# Create the chat model
def create_chat_model():
    chat_model = ChatWatsonx(
        model_id=model_id,
        url=credentials["url"],
        space_id=space_id,
        project_id=project_id,
        params=parameters,
        watsonx_client=client,
    )
    return chat_model

In [ ]:
from ibm_watsonx_ai.deployments import RuntimeContext

context = RuntimeContext(api_client=client)




def create_utility_agent_tool(tool_name, params, api_client, **kwargs):
    from langchain_core.tools import StructuredTool
    utility_agent_tool = Toolkit(
        api_client=api_client
    ).get_tool(tool_name)

    tool_description = utility_agent_tool.get("description")

    if (kwargs.get("tool_description")):
        tool_description = kwargs.get("tool_description")
    elif (utility_agent_tool.get("agent_description")):
        tool_description = utility_agent_tool.get("agent_description")
    
    tool_schema = utility_agent_tool.get("input_schema")
    if (tool_schema == None):
        tool_schema = {
            "type": "object",
            "additionalProperties": False,
            "$schema": "http://json-schema.org/draft-07/schema#",
            "properties": {
                "input": {
                    "description": "input for the tool",
                    "type": "string"
                }
            }
        }
    
    def run_tool(**tool_input):
        query = tool_input
        if (utility_agent_tool.get("input_schema") == None):
            query = tool_input.get("input")

        results = utility_agent_tool.run(
            input=query,
            config=params
        )
        
        return results.get("output")
    
    return StructuredTool(
        name=tool_name,
        description = tool_description,
        func=run_tool,
        args_schema=tool_schema
    )


def create_custom_tool(tool_name, tool_description, tool_code, tool_schema, tool_params):
    from langchain_core.tools import StructuredTool
    import ast

    def call_tool(**kwargs):
        tree = ast.parse(tool_code, mode="exec")
        custom_tool_functions = [ x for x in tree.body if isinstance(x, ast.FunctionDef) ]
        function_name = custom_tool_functions[0].name
        compiled_code = compile(tree, 'custom_tool', 'exec')
        namespace = tool_params if tool_params else {}
        exec(compiled_code, namespace)
        return namespace[function_name](**kwargs)
        
    tool = StructuredTool(
        name=tool_name,
        description = tool_description,
        func=call_tool,
        args_schema=tool_schema
    )
    return tool

def create_custom_tools():
    custom_tools = []


def create_tools(context):
    tools = []
    
    config = None
    tools.append(create_utility_agent_tool("GoogleSearch", config, client))
    config = {
    }
    tools.append(create_utility_agent_tool("DuckDuckGo", config, client))
    config = {
    }
    tools.append(create_utility_agent_tool("WebCrawler", config, client))
    config = {
        "maxResults": 5
    }
    tools.append(create_utility_agent_tool("Wikipedia", config, client))

    return tools

In [ ]:
def create_agent(context):
    # Initialize the agent
    chat_model = create_chat_model()
    tools = create_tools(context)

    memory = MemorySaver()
    instructions = """Always greet the user only once per session, even if they send multiple greetings.
Respond politely and helpfully, using a friendly and human-like tone.
After initial greeting, ask the user to choose a career area they want guidance in.
Based on user input, respond with structured, easy-to-understand information using bullet points or numbering.
Use emojis like 🎯 🧠 📚 ✨ 🙌 to keep responses engaging and readable.
Ask follow-up questions at each step to gather information about the user's skill level, daily available study time, and total learning duration.
After generating the roadmap, ask if they want help in another career area.
If the user says “Thanks” or “No Thanks,” respond with a warm closing and offer future support.
Always follow the latest user command strictly and do not mix or assume intent.
If the user asks about exams, internships, degrees, projects, or jobs—respond clearly and concisely with accurate information.
Encourage exploration but never overwhelm—keep guidance practical and motivating.
🔹 1. Greeting Logic
- When the user says “Hi”, “Hello”, or any greeting, \"Help me choose a tech career\" , \"Suggest a learning roadmap for AI/ML\" , \"How can I start a career in cybersecurity ?\"   respond with a warm welcome only once per session.
- Example:
  \"👋 Hello! I’m CaryoAI, your AI career guide. Ready to explore your career journey? Let’s get started!\"

🔹 2. Ask Career Interest
- After greeting, ask the user:
  \"Which career field would you like guidance in today? (e.g., Web Development, AI/ML, Cloud Computing, etc.)\"
- Wait for the user's response before continuing.

🔹 3. Respond to Career Field
- When the user gives a career field, do the following:
  1. Provide a short description of the field.
  2. Mention future benefits (scope, job roles, demand).
  3. Then ask:
     “What is your current skill level? (Beginner, Intermediate, Advanced)”
     “How many hours can you study daily? (1, 2, or 3+)”
     “How many months do you want to dedicate to learning this? (e.g., 1, 3, 6, 9 months, or 1 year)”

🔹 4. Generate Custom Roadmap
- Based on the career field, skill level, daily time, and total duration, generate a personalized roadmap.
- Include:
  - ✅ Major topics to cover
  - 🎯 Learning path (week-wise or month-wise)
  - 🧪 Projects to build
  - 🧾 Certifications to consider
  - 📚 Resources or platforms (Coursera, YouTube, etc.)
  - 🔁 Final tips or warnings based on user type (e.g., if they are beginners with only 1 hour/day)

🔹 5. Additional Conditions to Handle
- If the user asks:
  - \"What exams can I give?\" → Mention relevant entrance exams or certifications.
  - \"Are there internships available?\" → Guide them to platforms like Internshala, LinkedIn, etc.
  - \"Is there any degree/diploma for this field?\" → Mention popular academic paths and online options (BCA, MCA, PG Diploma, etc.).
  - \"What kind of jobs or roles can I apply for after this?\" → Give 3–5 job roles relevant to their track.

🔹 6. Final Interaction
- After sharing the roadmap:
  - Ask:
    \"Do you want guidance for another career field?\"
    If yes: Restart from career interest.
    If no or user says “Thanks” / “No Thanks”:
    Respond politely:
    \"You're most welcome! 😊 Wishing you the best in your career journey. Let me know if you need help anytime!\"

🔹 7. Response Format & Style (VERY IMPORTANT)
- Keep responses:
  - Structured using headings, bullets, or numbered lists
  - Friendly, human-like tone (avoid robotic repetition)
  - Add emoji for engagement: 🎯 🧠 📚 ✨ 🙌 where relevant

🔹 8. Final Instruction
Always respond based on the latest user input only. Do not mix up responses.
If the user says something unclear, ask a clarifying question.

🔹 9. Common Instruction
You are an intelligent, friendly, and structured career counseling assistant named \"CaryoAI\". Guide users in selecting, planning, and succeeding in tech careers based on their goals. Keep your tone human-like and helpful. Avoid generic or vague replies. Follow user flow and respond step-by-step based on user inputs. Do not ask for the same input again. Keep the experience smooth and professional.
"""

    agent = create_react_agent(chat_model, tools=tools, checkpointer=memory, state_modifier=instructions)

    return agent

In [ ]:
# Visualize the graph
from IPython.display import Image, display
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles

Image(
    create_agent(context).get_graph().draw_mermaid_png(
        draw_method=MermaidDrawMethod.API,
    )
)


## Invoking the agent
Let us now use the created agent, pair it with the input, and generate the response to your question:


In [ ]:
agent = create_agent(context)

def convert_messages(messages):
    converted_messages = []
    for message in messages:
        if (message["role"] == "user"):
            converted_messages.append(HumanMessage(content=message["content"]))
        elif (message["role"] == "assistant"):
            converted_messages.append(AIMessage(content=message["content"]))
    return converted_messages

question = input("Question: ")

messages = [{
    "role": "user",
    "content": question
}]

generated_response = agent.invoke(
    { "messages": convert_messages(messages) },
    { "configurable": { "thread_id": "42" } }
)

print_full_response = False

if (print_full_response):
    print(generated_response)
else:
    result = generated_response["messages"][-1].content
    print(f"Agent: {result}")


# Next steps
You successfully completed this notebook! You learned how to use
watsonx.ai inferencing SDK to generate response from the foundation model
based on the provided input, model id and model parameters. Check out the
official watsonx.ai site for more samples, tutorials, documentation, how-tos, and blog posts.

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2024 IBM. This notebook and its source code are released under the terms of the ILAN License.
Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for watsonx.ai Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for watsonx.ai Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="https://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF" target="_blank">License Terms</a>  